In [1]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import matplotlib.colors as mcolors

In [5]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
vehicle_matrix = pd.read_excel(repo_url + 'data/vhc_matrix_city_excluded.xlsx')
vehicle_matrix = vehicle_matrix.iloc[0:, :1]
vehicle_matrix
v = vehicle_matrix.values.tolist()
v = [item[0] for item in v]
full_data = pd.read_csv(repo_url + 'data/full_data.csv')